In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

url = 'https://www.pikalytics.com/pokedex/gen9vgc2023series2'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.find_all('table')

with open('pikalytics_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            writer.writerow([cell.text.strip() for cell in row.find_all('td')])x  Nh   

In [ ]:
# %% [code]
# %% [markdown] {"heading_collapsed":true}
# # 0. Setupcd

# %% [markdown] {"hidden":true}
# ## python version

# %% [code] {"ExecuteTime":{"end_time":"2022-05-19T22:39:39.172637Z","start_time":"2022-05-19T22:39:39.161634Z"},"hidden":true}
from platform import python_version
print(python_version())

# %% [markdown] {"hidden":true}
# ## Display Size

# %% [code] {"ExecuteTime":{"end_time":"2022-05-19T22:39:39.502970Z","start_time":"2022-05-19T22:39:39.479884Z"},"hidden":true}
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# %% [markdown] {"hidden":true}
# ## Librerie

# %% [code] {"ExecuteTime":{"end_time":"2022-05-19T22:39:41.466130Z","start_time":"2022-05-19T22:39:40.309145Z"},"hidden":true}
# Scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import mechanicalsoup as ms
import scrapy
import selenium
import lxml
import re
import requests
import pandas as pd
from pandas import compat
import json
from requests import get
import pprint

# Matching
import Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Record Linkage
import recordlinkage as rl
from recordlinkage.preprocessing import clean, phonetic
#import html5lib

# %% [markdown] {"hidden":true}
# ## Setup Selenium

# %% [code] {"ExecuteTime":{"end_time":"2022-05-19T22:39:41.527995Z","start_time":"2022-05-19T22:39:41.482844Z"},"hidden":true}
from selenium import webdriver
from selenium.webdriver.common.keys import Keys



# %% [markdown] {"heading_collapsed":true}
# # 2. `Pikalytics` Series 12 VGC 2022 (Apr 22)

# %% [markdown] {"hidden":true}
# ## Selenium

# %% [markdown] {"hidden":true}
# - *Lista dei pokemon registrati e link alle specifiche pagine*

# %% [code] {"ExecuteTime":{"end_time":"2022-04-23T21:47:43.578693Z","start_time":"2022-04-23T21:47:42.538916Z"},"hidden":true}
driver = webdriver.Edge('msedgedriver')

# %% [code] {"ExecuteTime":{"end_time":"2022-04-23T21:47:53.853973Z","start_time":"2022-04-23T21:47:52.615027Z"},"hidden":true}
driver.get('https://pikalytics.com/pokedex/ss/')

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T22:12:54.202899Z","start_time":"2022-04-20T22:12:54.188811Z"},"hidden":true}
driver.title

# %% [markdown] {"hidden":true}
# Seleziono la porzione di scermo di interesse

# %% [code] {"ExecuteTime":{"end_time":"2022-04-23T21:48:09.500316Z","start_time":"2022-04-23T21:48:09.467681Z"},"hidden":true}
pokemon_list = driver.find_element(by = 'xpath', value = '//*[@id="pokedex_wrapper"]')

# %% [markdown] {"hidden":true}
# Scrollo in modo tale da mostrare tutti i pokemon nell'HTML

# %% [code] {"ExecuteTime":{"end_time":"2022-04-23T21:48:10.430184Z","start_time":"2022-04-23T21:48:10.411928Z"},"hidden":true}
from selenium.webdriver.common.action_chains import ActionChains

# %% [markdown] {"hidden":true}
# Muovo il cursore sulla lista (non serve, funziona anche il tab da solo)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-23T21:48:17.319913Z","start_time":"2022-04-23T21:48:17.027612Z"},"hidden":true}
actions = ActionChains(driver)
actions.move_to_element(pokemon_list).perform()

# %% [markdown] {"hidden":true}
# Faccio tabbare un numero molto superiore al numero di pokemon in modo da assicurarmi che vengano mostrati tutti

# %% [code] {"ExecuteTime":{"end_time":"2022-04-23T21:49:54.751242Z","start_time":"2022-04-23T21:48:24.447836Z"},"hidden":true}
for i in range(1,10000):
    actions.key_down(value = '\ue004').perform()

# %% [markdown] {"hidden":true}
# Estraggo il HTMLResponse Object dalla pagina ora completamente caricata

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T17:32:07.370424Z","start_time":"2022-04-20T17:32:07.268791Z"},"hidden":true}
pikalytics_page = driver.page_source

# %% [code] {"ExecuteTime":{"start_time":"2022-04-23T21:49:58.405Z"},"hidden":true}
driver.close()
driver.quit()

# %% [markdown] {"hidden":true}
# ## bs4

# %% [markdown] {"hidden":true}
# - *Estraggo la lista dal HTTPResponse Object sottenuto con selenium*

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T17:35:31.919833Z","start_time":"2022-04-20T17:35:31.545187Z"},"hidden":true}
pikalytics_soup = BeautifulSoup(pikalytics_page, features='html.parser')

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T16:01:57.765361Z","start_time":"2022-04-21T16:01:57.761319Z"},"hidden":true,"scrolled":true}
#print(pikalytics_soup.prettify())

# %% [markdown] {"hidden":true}
# ### Lista dei link alle pagine dei singoli pokemon

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T17:41:58.265173Z","start_time":"2022-04-20T17:41:58.250087Z"},"hidden":true}
pikalytics_pkm_min_list_ul = pikalytics_soup.find(id = "min_list")

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T22:19:33.584495Z","start_time":"2022-04-20T22:19:33.567279Z"},"hidden":true}
pikalytics_pkm_min_list_a = pikalytics_pkm_min_list_ul.find_all('a', recursive=False)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T22:19:39.168103Z","start_time":"2022-04-20T22:19:39.149885Z"},"hidden":true}
pikalytics_pkm_min_list_links = []

# %% [code] {"ExecuteTime":{"end_time":"2022-04-20T22:19:40.574475Z","start_time":"2022-04-20T22:19:40.560380Z"},"hidden":true}
for i in pikalytics_pkm_min_list_a:
    pikalytics_pkm_min_list_links.append(i['href'])

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T16:02:05.868796Z","start_time":"2022-04-21T16:02:05.852457Z"},"hidden":true,"scrolled":true}
#pikalytics_pkm_min_list_links

# %% [markdown] {"hidden":true}
# ### Definizione Dataset Vuoti

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:10:07.735175Z","start_time":"2022-04-21T02:10:07.718951Z"},"hidden":true}
usage_S12VGC_columns = ['Pokemon', 'Monthly Usage (k)', 'Usage Percent (%)', 'Monthly Rank']
usage_S12VGC = pd.DataFrame(columns = usage_S12VGC_columns)

basestats_S12VGC_columns = ['Pokemon', 'HP', 'Atk', 'Def', 'SpA', 'SpD', 'Spe']
basestats_S12VGC = pd.DataFrame(columns = basestats_S12VGC_columns)

moves_S12VGC_columns = ['Pokemon', 'Move', 'Type', 'Use_Percentage (%)']
moves_S12VGC = pd.DataFrame(columns = moves_S12VGC_columns)
                            
teammates_S12VGC_columns = ['Pokemon', 'Teammate', 'Use_Percentage (%)']
teammates_S12VGC = pd.DataFrame(columns = teammates_S12VGC_columns)
                            
items_S12VGC_columns = ['Pokemon', 'Item', 'Use_Percentage (%)']
items_S12VGC = pd.DataFrame(columns = items_S12VGC_columns)
                            
abilities_S12VGC_columns = ['Pokemon', 'Ability', 'Use_Percentage (%)']
abilities_S12VGC = pd.DataFrame(columns = abilities_S12VGC_columns)
                            
EV_spread_S12VGC_columns = ['Pokemon', 'Nature', 'HP/Atk/Def/SpA/SpD/Spe', 'Use_Percentage (%)']
EV_spread_S12VGC = pd.DataFrame(columns = EV_spread_S12VGC_columns)

# %% [markdown] {"hidden":true}
# ### Scraping bs4

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:31.508088Z","start_time":"2022-04-21T02:10:08.843796Z"},"hidden":true,"scrolled":true}
from IPython.display import clear_output
lista_dei_vuoti = []
for i, link in enumerate(pikalytics_pkm_min_list_links):
    
    #print(link)
    
    # Acquisizione soup pagina specifica
    
    pikalytics_pkm_URL = "https://pikalytics.com" + link
    pikalytics_pkm_page = requests.get(pikalytics_pkm_URL)
    pikalytics_pkm_soup = BeautifulSoup(pikalytics_pkm_page.content,
                                        features='html.parser')
    
    ## Controllo che la pagina esista verificando che compaia il nome del pokemon
    if (pikalytics_pkm_soup.find('h1') == None):
        lista_dei_vuoti.append(link)
        continue
    
    pikalytics_pkm_name = (pikalytics_pkm_soup.find('h1').text)

    # Avanzamento
    print(
        f'Avanzamento: {i}/{len(pikalytics_pkm_min_list_links)} - Pokemon: {pikalytics_pkm_name}'
    )

    # Usage Stats

    pikalytics_pkm_usagestats_temp = [f'{pikalytics_pkm_name}']
    pikalytics_pkm_usagestats_div = pikalytics_pkm_soup.find(
        'div', class_="header-div-right header-div-right-margin").find_all(
            'div', class_='pokemon-ind-summary-text')
    ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
    if (len(pikalytics_pkm_usagestats_div) < 3):
        pikalytics_pkm_usagestats_temp = [f'{pikalytics_pkm_name}', '-', '-', '-']
    else:
        for e, stat in enumerate(pikalytics_pkm_usagestats_div):
            if (e == 2):
                pikalytics_pkm_usagestats_temp.append(stat.contents[1])
            else:
                pikalytics_pkm_usagestats_temp.append(stat.contents[0])
    ## Trasformo riga in serie e appendo
    usagestats_series = pd.Series(pikalytics_pkm_usagestats_temp,
                                  index=usage_S12VGC.columns)
    usage_S12VGC = usage_S12VGC.append(usagestats_series, ignore_index=True)

    # Base Stats

    pikalytics_pkm_basestats_temp = [f'{pikalytics_pkm_name}']
    pikalytics_pkm_basestats_stat_wrapper = pikalytics_pkm_soup.find(
        'div', id='bstats_wrapper').find('div',
                                         recursive=False).find('div',
                                                               recursive=False)
    ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
    if pikalytics_pkm_basestats_stat_wrapper:
        pikalytics_pkm_basestats_stat_div = pikalytics_pkm_basestats_stat_wrapper.find_all(
            'div', recursive=False)
        for stat in pikalytics_pkm_basestats_stat_div:
            pikalytics_pkm_basestats_temp.append(
                stat.find_all('div', recursive=False)[1].text)
        ## Trasformo riga in serie e appendo
        base_stat_series = pd.Series(pikalytics_pkm_basestats_temp,
                                     index=basestats_S12VGC.columns)
        basestat_S12VGC = basestat_S12VGC.append(base_stat_series,
                                                 ignore_index=True)
    # Moves Stats

    pikalytics_pkm_moves_wrapper = pikalytics_pkm_soup.find(id="moves_wrapper")
    pikalytics_pkm_moves_div = pikalytics_pkm_moves_wrapper.find_all(
        'div', class_="pokedex-move-entry-new")
    for moves in pikalytics_pkm_moves_div:
        # In questo caso, essendoci più mosse consigliate per ogni pokemon, dobbiamo creare una lista per ogni mossa (una riga nel dataframe)
        pikalytics_pkm_moves_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not moves.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 3 variabili
        move = moves.find_all('div', recursive=False)
        pikalytics_pkm_moves_temp.append(move[0].text)
        pikalytics_pkm_moves_temp.append(move[1].find('span').text)
        pikalytics_pkm_moves_temp.append(move[2].text)
        ## Trasformo riga in serie e appendo
        moves_series = pd.Series(pikalytics_pkm_moves_temp,
                                 index=moves_S12VGC.columns)
        moves_S12VGC = moves_S12VGC.append(moves_series, ignore_index=True)

    # Teammates Stats

    pikalytics_pkm_teammates_wrapper = pikalytics_pkm_soup.find(
        id="dex_team_wrapper")
    pikalytics_pkm_teammates_div = pikalytics_pkm_teammates_wrapper.find_all(
        'a', class_="teammate_entry", recursive=False)
    for teammates in pikalytics_pkm_teammates_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_teammates_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not teammates.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 2 variabili
        pikalytics_pkm_teammates_temp.append(teammates['data-name'])
        pikalytics_pkm_teammates_temp.append(
            teammates.find_all('div', recursive=False)[3].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        teammates_series = pd.Series(pikalytics_pkm_teammates_temp,
                                     index=teammates_S12VGC.columns)
        teammates_S12VGC = teammates_S12VGC.append(teammates_series,
                                                   ignore_index=True)

    # Items Stats

    pikalytics_pkm_items_wrapper = pikalytics_pkm_soup.find(
        id="items_wrapper").find('div')
    pikalytics_pkm_items_div = pikalytics_pkm_items_wrapper.find_all(
        'div', recursive=False)
    for item in pikalytics_pkm_items_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_items_temp = [f'{pikalytics_pkm_name}']
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not item.find_all('div', recursive=False):
            break
        # Vado ad aggiungere separatamente le 2 variabili
        pikalytics_pkm_items_temp.append(
            item.find_all('div', recursive=False)[1].text)
        pikalytics_pkm_items_temp.append(
            item.find_all('div', recursive=False)[2].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        items_series = pd.Series(pikalytics_pkm_items_temp,
                                 index=items_S12VGC.columns)
        items_S12VGC = items_S12VGC.append(items_series, ignore_index=True)
    
    # Abilities

    pikalytics_pkm_abilities_wrapper = pikalytics_pkm_soup.find(
        id="abilities_wrapper").find('div')
    pikalytics_pkm_abilities_div = pikalytics_pkm_abilities_wrapper.find_all(
        'div', recursive=False)
    for ability in pikalytics_pkm_abilities_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_abilities_temp = [f'{pikalytics_pkm_name}']
        # Vado ad aggiungere separatamente le 2 variabili
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not ability.find_all('div', recursive=False):
            break
        pikalytics_pkm_abilities_temp.append(
            ability.find_all('div', recursive=False)[0].text)
        pikalytics_pkm_abilities_temp.append(
            ability.find_all('div', recursive=False)[1].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_teammates_temp)
        abilities_series = pd.Series(pikalytics_pkm_abilities_temp,
                                 index=abilities_S12VGC.columns)
        abilities_S12VGC = abilities_S12VGC.append(abilities_series, ignore_index=True)
        
    # ev_spread

    pikalytics_pkm_ev_spread_wrapper = pikalytics_pkm_soup.find(
        id="dex_spreads_wrapper").find('div')
    pikalytics_pkm_ev_spread_div = pikalytics_pkm_ev_spread_wrapper.find_all(
        'div', recursive=False)
    for ev_spread in pikalytics_pkm_ev_spread_div:
        # In questo caso, essendoci più teammates consigliaie per ogni pokemon, dobbiamo creare una lista per ogni teammate (una riga nel dataframe)
        pikalytics_pkm_ev_spread_temp = [f'{pikalytics_pkm_name}']
        # Vado ad aggiungere separatamente le 3 variabili
        ## Prima controllo che ci sia la tabella (Alcune volte i dati non sono disponibili)
        if not ev_spread.find_all('div', recursive=False):
            break
        ## Natura
        pikalytics_pkm_ev_spread_temp.append(
            ev_spread.find_all('div', recursive=False)[0].text)
        ## Spreads li vado a sommare per ottenere un'unica stat
        spread_div = ev_spread.find_all('div', recursive=False)[1:7]
        spread_temp = ""
        for ev in (ev_spread.find_all('div', recursive=False)[1:7]):
            spread_temp += ev.text
        pikalytics_pkm_ev_spread_temp.append(spread_temp)
        ## Percentage
        pikalytics_pkm_ev_spread_temp.append(
            ev_spread.find_all('div', recursive=False)[7].text)
        ## Trasformo riga in serie e appendo
        #print(pikalytics_pkm_ev_spread_temp)
        ev_spread_series = pd.Series(pikalytics_pkm_ev_spread_temp,
                                 index=EV_spread_S12VGC.columns)
        EV_spread_S12VGC = EV_spread_S12VGC.append(ev_spread_series, ignore_index=True)
    clear_output(wait=True)
    #if (i > 5):
    #    break

# %% [markdown] {"heading_collapsed":true,"hidden":true}
# ## Dataset Ottenuti

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T16:02:36.229441Z","start_time":"2022-04-21T16:02:36.226358Z"},"hidden":true}
#lista_dei_vuoti

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:33.856422Z","start_time":"2022-04-21T02:20:33.844418Z"},"hidden":true}
display(usage_S12VGC)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:35.113975Z","start_time":"2022-04-21T02:20:35.099912Z"},"hidden":true}
display(basestat_S12VGC)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:36.387882Z","start_time":"2022-04-21T02:20:36.373712Z"},"hidden":true}
display(moves_S12VGC)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:37.733709Z","start_time":"2022-04-21T02:20:37.720362Z"},"hidden":true,"scrolled":true}
display(teammates_S12VGC)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:39.131523Z","start_time":"2022-04-21T02:20:39.102532Z"},"hidden":true}
display(items_S12VGC)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:40.438180Z","start_time":"2022-04-21T02:20:40.408669Z"},"hidden":true}
display(abilities_S12VGC)

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:41.640338Z","start_time":"2022-04-21T02:20:41.614055Z"},"hidden":true}
display(EV_spread_S12VGC)

# %% [markdown] {"hidden":true}
# ## Salvataggio .csv relazionali

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T02:20:42.917173Z","start_time":"2022-04-21T02:20:42.841021Z"},"hidden":true}
usage_S12VGC.to_csv('./Scraping_datasets/pikalytics/usage_S12VGC.csv', na_rep="NA")
basestat_S12VGC.to_csv('./Scraping_datasets/pikalytics/basestat_S12VGC.csv', na_rep="NA")
moves_S12VGC.to_csv('./Scraping_datasets/pikalytics/moves_S12VGC.csv', na_rep="NA")
teammates_S12VGC.to_csv('./Scraping_datasets/pikalytics/teammates_S12VGC.csv', na_rep="NA")
items_S12VGC.to_csv('./Scraping_datasets/pikalytics/items_S12VGC.csv', na_rep="NA")
abilities_S12VGC.to_csv('./Scraping_datasets/pikalytics/abilities_S12VGC.csv', na_rep="NA")
EV_spread_S12VGC.to_csv('./Scraping_datasets/pikalytics/EV_spread_S12VGC.csv', na_rep="NA")

# %% [markdown] {"hidden":true}
# ## Salvataggio .JSON dizionari

# %% [code] {"ExecuteTime":{"end_time":"2022-04-21T13:29:04.888803Z","start_time":"2022-04-21T13:29:02.974624Z"},"hidden":true}
S12VGC_df = {}
S12VGC_json = {}
dataset_list = ['usage_S12VGC', 'basestat_S12VGC', 'moves_S12VGC', 'teammates_S12VGC', 'items_S12VGC', 'abilities_S12VGC', 'EV_spread_S12VGC']

for dataset_name in dataset_list:
    print(dataset_name)
    S12VGC_df[f'{dataset_name}'] = pd.read_csv(f'./Scraping_datasets/pikalytics/{dataset_name}.csv', na_values = 'NA').iloc[: , 1:]
    S12VGC_json[f'{dataset_name}'] = json.dumps(list(S12VGC_df[f'{dataset_name}'].agg(lambda x: x.dropna().to_dict(), axis=1)), indent = 4, ensure_ascii=False)
    with open(f'./Scraping_datasets/pikalytics/{dataset_name}.json', 'w', encoding='utf8') as outfile:
        outfile.write(S12VGC_json[f'{dataset_name}'])


#

In [1]:
import sys
!{sys.executable} -m pip install 'type here what you would like to install'

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
